In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/home/baldeeb/Code/pytorch-NOCS')

from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_Weights
from torch.utils.data import DataLoader
from utils.load_save import load_nocs
from torch.optim import Adam
import torch

from habitat_datagen_util.utils.dataset import HabitatDataset
from habitat_datagen_util.utils.collate_tools import collate_fn

from utils.spot_dataset import SpotDataset, collate_fn 

DATA_DIR = '/home/baldeeb/Code/bd_spot_wrapper/data/output/front_left'
CHKPT_DIR = "/home/baldeeb/Code/pytorch-NOCS/checkpoints/nocs_classification/2023-05-25_13-46-18/one_class_frozen_backbone_4.pth"
device= 'cuda:1'

model = load_nocs(CHKPT_DIR,
                maskrcnn_weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT,
                nocs_num_bins=32,
                nocs_loss_mode = 'classification',
                multiheaded_nocs = True,)
model.to(device).train()

habitatdata = SpotDataset(DATA_DIR)
dataloader = DataLoader(habitatdata, 
                        batch_size=2, 
                        collate_fn=collate_fn)
optimizer = Adam(model.parameters(keys=['nocs']))

In [2]:
import torch
poses = []
for im, tgt in dataloader:
    poses.append(tgt[0]['camera_pose'])
poses = torch.stack(poses)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for i in range(len(poses)-2):
    plt.figure()
    pose = poses[i:i+2]
    a = [1/v for v in range(1, len(pose)+1)]
    plt.plot(pose[0:1, 0,3], pose[0:1, 2,3], alpha=1)
    plt.plot(pose[1:2, 0,3], pose[1:2, 2,3], alpha=0.5)
    plt.show()
    plt.close()

In [ ]:
model.roi_heads.training_mode('multiview')
for itr, (images, targets) in enumerate(dataloader):
    images = images.to(device)
    # targets = targets2device(targets, device)
    losses = model(images, targets)
    loss = sum(losses.values())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(losses['multiview_consistency_loss'])
